# Support Vector Machine

In [91]:
import warnings
warnings.filterwarnings('ignore')

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

## SVC(classification) Class 

hyperplane을 얻는데 필요한 Convex 최적화 알고리즘을 제공하는 cvxopt를 사용하였다.  
SVC(support vector classification)클래스를 생성하였다. kernel은 rbf, polynomial, sigmoid를 사용하였으며 세가지 모두 사용하지 않는다면 Linear & Soft margin을 사용하도록 설정했다. 

전처리를 위한 함수로 make_label_map 함수, 원 데이터의 라벨을 실제로 1, -1로 변환하는 transform_label 함수, 1과 -1을 원 데이터의 라벨로 바꿔주는 inverse_label 함수, 두 벡터의 kernel 값을 계산하는 get_kernel_val 함수를 생성하였다. 

위 함수에서 X, y 가 정의되면 P, q, G, h, A, b를 numpy array 형식으로 작성하였으며 형식은 다음과 같다. 

$$
\begin{align} x &=\alpha\;\; (n \times 1 \; \text{vector}), \\ P&=K \;\;(n\times n \; \text{matrix}), \\ q &= -e \;\; (n \times 1 \; \text{vector}), \\ G &=\begin{pmatrix} -I \\ I \end{pmatrix} \;\;  (n \times n \; \text{matrix}), \\ h &= (0, \ldots, 0, C, \ldots, C)^t \;\; (2n \times 1 \;\text{vector}), \\ A &= y^t \;\;(1\times n \;\text{matrix}), \\ b &= 0 \;\;(\text{scalar}) \end{align}
$$


In [93]:
class SVC():
    def __init__(self, kernel=None, gamma = None, coef0=0):
        self.labels_map = None
        self.kernel = kernel
        self.X = None
        self.y = None
        self.alphas = None
        self.w = None
        self.b = None
        self.gamma = gamma
        self.coef0 = coef0
        if kernel is not None:
            assert kernel in [ 'rbf','polynomial', 'sigmoid']
        
    def fit(self, X, y):
        uniq_labels = np.unique(y)
        assert len(uniq_labels) == 2
        
            
        self.make_label_map(uniq_labels)
        self.X = X
        
        if not self.gamma:
            gamma = 1/(X.shape[1]*X.var())
        
        y = [self.transform_label(label, self.labels_map) for label in y] ## 1, -1로 변환
        y = np.array(y)
        ## formulating standard form
        m, n = X.shape
        y = y.reshape(-1,1)*1.
        self.y = y
        if self.kernel is not None:
            K = np.zeros((m,m))
            for i in range(m):
                for j in range(m):
                    K[i][j] = y[i]*y[j]*self.get_kernel_val(X[i], X[j])                       
        else:
            yX = y*X
            K = np.dot(yX,yX.T)
        P = cvxopt_matrix(K)
        q = cvxopt_matrix(-np.ones((m, 1)))
        G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
        h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
        A = cvxopt_matrix(y.reshape(1, -1))
        b = cvxopt_matrix(np.zeros(1))
        ## cvxopt configuration
        cvxopt_solvers.options['show_progress'] = False ## 결과 출력 X
        sol = cvxopt_solvers.qp(P, q, G, h, A, b) 
        alphas = np.array(sol['x'])
        S = (alphas>1e-4).flatten()
        
        
        if self.kernel is not None:
            sum_val = 0
            S_index = np.where(S==True)[0]
            for s in S_index:
                temp_vec = np.array([self.get_kernel_val(z, X[s]) for z in X])
                temp_vec = np.expand_dims(temp_vec, axis=1)
                sum_val += np.sum(y[s] - np.sum(y*alphas*temp_vec))
            b = sum_val/len(S)
            self.b = b
        else:
            w = ((y*alphas).T@X).reshape(-1,1)
            b = np.mean(y[S] - np.dot(X[S],w))
            self.w = w
            self.b = b
        self.alphas = alphas
        
        return
    
    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        
        return predictions
    
    def _predict(self, x):
        if self.kernel:
            temp_vec = np.array([self.get_kernel_val(x, y) for y in self.X])
            temp_vec = np.expand_dims(temp_vec, axis=1)
            S = (self.alphas>1e-4).flatten()
            res = np.sign(np.sum(self.y[S]*self.alphas[S]*temp_vec[S])+self.b)
        else:
            res = np.sign(self.w.T.dot(x)+self.b)
        res = self.inverse_label(res, self.labels_map)
        
        return res
    
    def make_label_map(self, uniq_labels):
        labels_map = list(zip([-1, 1], uniq_labels))
        self.labels_map = labels_map
        
        return
    
    def transform_label(self, label, labels_map):
        res = [l[0] for l in labels_map if l[1] == label][0]
        
        return res

    def inverse_label(self, svm_label, labels_map):
        try:
            res = [l[1] for l in labels_map if l[0] == svm_label][0]
        except:
            print(svm_label)
            print(labels_map)
            raise
        
        return res
    
    def get_kernel_val(self, x, y):
        X = self.X
        coef0 = self.coef0
        if not self.gamma:
            gamma = 1/(X.shape[1]*X.var())
            
        if self.kernel == 'rbf':
            return np.exp(-gamma*np.square(np.linalg.norm(x-y)))
        elif self.kernel == 'polynomial':
            return (gamma*np.dot(x,y)+coef0)**2
        else:
            return np.tanh(gamma*np.dot(x,y)+coef0)

## SVM class

SVM클래스는 분류와 회귀를 동시에 수행할 수 있도록 구현하였다. type을 통해서 classification 혹은 regression을 실행하도록 하였다. 마찬가지로 두 벡터에 대한 kernel값을 계산하는 함수가 필요하다. 
SVC는 classification을 위한 함수, SVR은 regression을 위한 함수로 구성하였다. 
SVC에서는 앞선 SVC 클래스에서 모형을 적합하여 SVM의 model_list에 저장한다. 

cvxopt에서 계산하는 식은 다음과 같다. 

$$
\begin{align} x &=(\alpha^t, \alpha^{*t})^t \;\; (2n \times 1 \; \text{vector}), \\ P&=(I -I)^tQ(I -I) \;\;(2n\times 2n \; \text{matrix}), \\ q &= \epsilon e_{2n} - (y^t, -y^t)^t \;\; (2n \times 1 \; \text{vector}), \\ G &=\begin{pmatrix} -I & O \\ I & O \\ O & -I \\ O & I \end{pmatrix} \;\;  (4n \times 2n \; \text{matrix}), \\ h &= (0, \ldots, 0, C, \ldots, C, 0, \ldots, 0, C, \ldots, C)^t \;\; (4n \times 1 \;\text{vector}), \\ A &= e_n^t(I -I) \;\;(1\times 2n \;\text{matrix}), \\ b &= 0 \;\;(\text{scalar}) \end{align}
$$

predict, predict_reg 함수를 통해 예측을 하고 Accuracy를 통해 학습 정확도를 계산하였다. 



In [94]:
class SVM():
    def __init__(self, type='classification', kernel=None, gamma=None, C=None, coef0=0):
        assert type in ['classification', 'regression']
        self.type = type
        self.kernel=kernel
        self.X = None
        self.y = None
        self.coef0 = coef0
        self.model_list = None
        self.gamma = gamma
        self.C = C
        if kernel is not None:
            assert kernel in ['polynomial', 'rbf', 'sigmoid']
            
    def fit(self, X, y, epsilon=[0.1, 0.01, 0.001]):
        if self.type == 'classification':
            self._fit_svc(X, y)
        else:
            self._fit_svr(X, y, epsilon)
            
    def _fit_svc(self, X, y):
        uniq_labels = np.unique(y)
        label_combinations = list(combinations(uniq_labels, 2))
        model_list = []
        for lc in label_combinations:
            target_idx = np.array([x in lc for x in y])
            y_restricted = y[target_idx]
            X_restricted = X[target_idx]
            clf = SVC(kernel=self.kernel, coef0=self.coef0)
            clf.fit(X_restricted, y_restricted)
            model_list.append(clf)
        
        self.model_list = model_list
        return
    
    def _fit_svr(self, X, y, epsilon):

        assert epsilon > 0
        self.X = X
        m, n = X.shape
        y = y.reshape(-1,1)*1.
        self.y = y
        if self.kernel is not None:
            K = np.zeros((m,m))
            for i in range(m):
                for j in range(m):
                    K[i][j] = self.get_kernel_val(X[i], X[j])
        else:
            K = X.dot(X.T)
        I = np.eye(m)
        O = np.zeros((m, m))
        sub_K = np.hstack([I, -I])
        main_K = sub_K.T.dot(K.dot(sub_K))
        P = cvxopt_matrix(main_K)
        q = cvxopt_matrix(epsilon*np.ones((2*m, 1)) - np.vstack([y, -y]))

        G = np.vstack([np.hstack([-I, O]), np.hstack([I, O]), np.hstack([O, -I]), np.hstack([O, I])])
        G = cvxopt_matrix(G)
        h = cvxopt_matrix(np.hstack([np.zeros(m), C*np.ones(m)]*2))
        A = cvxopt_matrix(np.ones((m,1)).T.dot(sub_K))
        b = cvxopt_matrix(np.zeros(1))
        
        cvxopt_solvers.options['show_progress'] = False
        sol = cvxopt_solvers.qp(P, q, G, h, A, b)
        sol_root = np.array(sol['x'])
        alphas = sol_root[:m]
        alphas_star = sol_root[m:]
        
        S = (alphas>1e-4).flatten()       
        if self.kernel is not None:
            sum_val = []
            S_index = np.where(S==True)[0]
            for s in S_index:
                temp_vec = np.array([self.get_kernel_val(z, X[s]) for z in X])
                temp_vec = np.expand_dims(temp_vec, axis=1)
                sum_val.append(-epsilon + np.sum(y[s] - np.sum((alphas-alphas_star)*temp_vec)))
            b = min(sum_val)
            self.b = b

        else:
            w = alphas.T.dot(X)-alphas_star.T.dot(X)
            w = w.reshape(-1,1)
            b = -epsilon+np.min(y[S] - np.dot(X[S],w))
            self.w = w
            self.b = b
        self.alphas = sol_root
        
        return
    
    def predict(self, X):
        if self.type =='classification':
            model_list = self.model_list
            prediction = [model.predict(X) for model in model_list]
            prediction = [Counter(pred).most_common(1)[0][0] for pred in list(zip(*prediction))]
        else:
            prediction = [self._predict_reg(x) for x in X]
        return prediction
    
    def _predict_reg(self, x):
        X = self.X
        
        if self.kernel is not None:
            m, _ = self.X.shape
            sol_root = self.alphas
            alphas = sol_root[:m]
            alphas_star = sol_root[m:]
            
            temp_vec = np.array([self.get_kernel_val(z, x) for z in X])
            temp_vec = np.expand_dims(temp_vec, axis=1)
            pred = np.sum((alphas-alphas_star)*temp_vec)+self.b
        else:
            w = self.w
            b = self.b
            pred = w.dot(x)+b
            pred = pred[0]
        return pred
    
    def get_kernel_val(self, x, y):
        X = self.X
        coef0 = self.coef0
        
        if not self.gamma:
            gamma = 1/(X.shape[1]*X.var())
            
        if self.kernel == 'rbf':
            return np.exp(-gamma*np.square(np.linalg.norm(x-y)))
        elif self.kernel == 'polynomial':
            return (gamma*np.dot(x,y)+coef0)**2 
        elif self.kernel == 'sigmoid':
            return np.tanh(gamma*np.dot(x,y)+coef0)
        

### Data
분류를 위한 데이터는 붓꽃 데이터를 이용하였다. 

In [95]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = [iris.target_names[x] for x in iris.target]
 
species_to_labels = dict(zip(df['species'].unique(), range(len(df['species'].unique()))))
df['species'] = df['species'].map(species_to_labels)
df = df.rename(columns={'species':'label'})

X = df.drop('label', axis=1).values
y = df['label'].values


## Linear & Soft margin
C값에 따른 차이를 보기 위해 C = [0.1, 1, 10, 100, 1000]으로 Accuracy값을 확인하였다. 
확인 결과 C값에 따른 정확도의 차이는 거의 없다고 봐도 무방한 것 같다.  

마진 관점에서는 C값이 클수록 $/xi$값이 작아지고 마진이 좁아지며, C값이 작을수록 $/xi$가 커지고 마진도 커진다. 데이터때문인가 마진 값이 정확도에 미치는 영향을 알아보기 위해서는 다른 데이터를 사용하거나 이미지를 생성해봐야 알거 같다. 

In [103]:
for kernel in [None]:
    for gamma in [None]:
        for C in [0.1, 1, 10, 100, 1000]:
            clf_linear = SVM(type='classification', kernel=None, gamma=None, C=None)
            clf_linear.fit(X, y)
            pred_linear = clf_linear.predict(X)
            print('kernel: Linear', 'C:', C, 'Accuracy:', np.mean(y==pred_linear))

kernel: Linear C: 0.1 Accuracy: 0.98
kernel: Linear C: 1 Accuracy: 0.9866666666666667
kernel: Linear C: 10 Accuracy: 0.98
kernel: Linear C: 100 Accuracy: 0.98
kernel: Linear C: 1000 Accuracy: 0.9866666666666667


## Nonlinear
Kernel은 [rbf, polynomial, sigmoid] 3가지를 사용하였으며, gamma값은 [0.001, 0.01, 0.1, 1] 4가지, C값은 linear와 마찬가지로 [0.1, 1, 10, 100, 1000] 5가지를 사용하였다. 

결과는 다음과 같이 kernel은 rbf일때 C값은 10일때 Accuracy가 가장 높았으며, gamma값은 크게 영향을 받지 않는 것으로 확인했다. 그 이유는 iris data가 보다 단순한 편에 속하기 때문인 것 같다. 

이외에도 kernel이 polynomial, sigmoid인 경우에는 Accuracy가 높지 않았다. 따라서 성능에 영향을 미치는 우선순위를 고려한다면 Kernel > C > gamma 인 것 같다.


| Kernel | C | Gamma | Accuracy |
| :------: | :-: | :-----: | :--------: |
| rbf | 10 | 0.001 | 0.987 |
| rbf | 10 | 0.01 | 0.987 |
| rbf | 10 | 0.1 | 0.987 |
| rbf | 10 | 1 | 0.987 |



In [96]:
for kernel in ['rbf', 'polynomial', 'sigmoid']:
    for gamma in [0.001, 0.01, 0.1, 1]:
        for C in [0.1, 1, 10, 100, 1000]:
            clf = SVM(type='classification',kernel = kernel, gamma= gamma, C=C)
            clf.fit(X, y)
            pred = clf.predict(X)
            print('kernel:', kernel, 'C:', C, 'gamma:', gamma, 'Accuracy:', np.mean(y==pred))

kernel: rbf C: 0.1 gamma: 0.001 Accuracy: 0.9466666666666667
kernel: rbf C: 1 gamma: 0.001 Accuracy: 0.9666666666666667
kernel: rbf C: 10 gamma: 0.001 Accuracy: 0.9866666666666667
kernel: rbf C: 100 gamma: 0.001 Accuracy: 0.9333333333333333
kernel: rbf C: 1000 gamma: 0.001 Accuracy: 0.84
kernel: rbf C: 0.1 gamma: 0.01 Accuracy: 0.9466666666666667
kernel: rbf C: 1 gamma: 0.01 Accuracy: 0.9666666666666667
kernel: rbf C: 10 gamma: 0.01 Accuracy: 0.9866666666666667
kernel: rbf C: 100 gamma: 0.01 Accuracy: 0.9333333333333333
kernel: rbf C: 1000 gamma: 0.01 Accuracy: 0.84
kernel: rbf C: 0.1 gamma: 0.1 Accuracy: 0.9466666666666667
kernel: rbf C: 1 gamma: 0.1 Accuracy: 0.9666666666666667
kernel: rbf C: 10 gamma: 0.1 Accuracy: 0.9866666666666667
kernel: rbf C: 100 gamma: 0.1 Accuracy: 0.9333333333333333
kernel: rbf C: 1000 gamma: 0.1 Accuracy: 0.84
kernel: rbf C: 0.1 gamma: 1 Accuracy: 0.9466666666666667
kernel: rbf C: 1 gamma: 1 Accuracy: 0.9666666666666667
kernel: rbf C: 10 gamma: 1 Accuracy:

# Support Vector Regression

SVR은 sklearn의 boston data를 사용하였다. SVM에서 type = regression으로 적용 한 뒤 모형을 적합시키고 MSE를 계산하여 비교하였다. 

In [104]:
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['y'] = boston.target

In [105]:
X = df[['LSTAT']].values
y = df['y'].values

SVR도 마찬가지로 kernel은 [rbf, polynomial], epsilon: [1, 0.1, 0.01, 0.001, 0.0001] 을 사용하여 각각 비교하였다. 

그 결과중 가장 성능이 좋은 것은 다음과 같. 

| Kernel | C | epsilon | MSE |
| :------: | :-: | :-----: | :--------: |
| rbf | 100 | 1 | 27.669 |
| rbf | 100 | 0.1 | 27.968 |
| rbf | 1000 | 0.01 | 27.998 |

kernel은 SVC와 마찬가지로 rbf일때가 가장 성능이 좋았으며 C값은 100이나 1000이었으며, epsilon은 1, 0.1, 0.01으로 모두 다른 값이었다. 따라서 SVC와 유사하게 Kernel > C > epsilon 순으로 성능에 영향을 많이 끼치는 것으로 볼 수 있다. 

In [110]:
for kernel in ['rbf','polynomial']:
    for C in [0.1, 1, 10, 100, 1000]:
        reg1 = SVM(type='regression', kernel=kernel, C=C)
        reg1.fit(X, y, epsilon= 1)
        pred = reg1.predict(X)
        mse = np.mean(np.square(y-pred))
        print('Kernel:', kernel, 'epsilon: 1', 'C:', C, 'MSE:', mse)
        
for kernel in ['rbf','polynomial']:
    for C in [0.1, 1, 10, 100, 1000]:
        reg2 = SVM(type='regression', kernel=kernel, C=C)
        reg2.fit(X, y, epsilon= 0.1)
        pred = reg2.predict(X)
        mse = np.mean(np.square(y-pred))
        print('Kernel:', kernel, 'epsilon: 0.1', 'C:', C, 'MSE:', mse)

for kernel in ['rbf','polynomial']:
    for C in [0.1, 1, 10, 100, 1000]:
        reg3 = SVM(type='regression', kernel=kernel, C=C)
        reg3.fit(X, y, epsilon= 0.01)
        pred = reg3.predict(X)
        mse = np.mean(np.square(y-pred))
        print('Kernel:', kernel, 'epsilon: 0.01', 'C:', C, 'MSE:', mse)

for kernel in ['rbf','polynomial']:
    for C in [0.1, 1, 10, 100, 1000]:
        reg4 = SVM(type='regression', kernel=kernel, C=C)
        reg4.fit(X, y, epsilon= 0.001)
        pred = reg4.predict(X)
        mse = np.mean(np.square(y-pred))
        print('Kernel:', kernel, 'epsilon: 0.001', 'C:', C, 'MSE:', mse)

for kernel in ['rbf','polynomial']:
    for C in [0.1, 1, 10, 100, 1000]:
        reg5 = SVM(type='regression', kernel=kernel, C=C)
        reg5.fit(X, y, epsilon= 0.0001)
        pred = reg5.predict(X)
        mse = np.mean(np.square(y-pred))
        print('Kernel:', kernel, 'epsilon: 0.0001', 'C:', C, 'MSE:', mse)        


Kernel: rbf epsilon: 1 C: 0.1 MSE: 50.91524022866217
Kernel: rbf epsilon: 1 C: 1 MSE: 32.904018201060474
Kernel: rbf epsilon: 1 C: 10 MSE: 28.19950942493443
Kernel: rbf epsilon: 1 C: 100 MSE: 27.66869241256815
Kernel: rbf epsilon: 1 C: 1000 MSE: 30.25441538874462
Kernel: polynomial epsilon: 1 C: 0.1 MSE: 56.41296989200502
Kernel: polynomial epsilon: 1 C: 1 MSE: 56.24795479413233
Kernel: polynomial epsilon: 1 C: 10 MSE: 56.247949120244414
Kernel: polynomial epsilon: 1 C: 100 MSE: 56.247950366874264
Kernel: polynomial epsilon: 1 C: 1000 MSE: 58.51204426380987
Kernel: rbf epsilon: 0.1 C: 0.1 MSE: 52.384773054323496
Kernel: rbf epsilon: 0.1 C: 1 MSE: 32.96065393164595
Kernel: rbf epsilon: 0.1 C: 10 MSE: 28.329411140878886
Kernel: rbf epsilon: 0.1 C: 100 MSE: 27.96831224640013
Kernel: rbf epsilon: 0.1 C: 1000 MSE: 86.89142735661936
Kernel: polynomial epsilon: 0.1 C: 0.1 MSE: 56.34479093618372
Kernel: polynomial epsilon: 0.1 C: 1 MSE: 56.32599477063427
Kernel: polynomial epsilon: 0.1 C: 10 M